# Example Notebook

This notebook shows the estimation of the parameters of 1-d AR(1) process:

$$ z_{t} = \delta + \rho z_{t-1} + \sigma e_{t} $$

We know that :
* the mean of $\{z_t\}$ is $\frac{\delta}{1 - \rho}$
* its variance is $\frac{\sigma^2}{1 - \rho^2}$
* its first order autocorrelation is $\rho$

The econometrician observes the following empirical moments:
* empirical mean of $z_{t} = 1.0$
* empirical variance of $z_{t} = 5.0$
* empirical autocorrelation of $z_{t} = \sqrt{0.8}$

The SMM estimated values should be very close to:
* $\hat{delta} = 1 - \rho \approx 0.1055$
* $\hat{rho} = \sqrt{0.8} \approx 0.8944 $
* $\hat{sigma} = \sqrt{(1.0 - \rho^2) \times \text{empirical variance of } z_t } = 1$



In [1]:
using SMM
using Distributions
using DataStructures

# Because of this issue (https://github.com/JuliaIO/JLD2.jl/issues/107)
# you also need to import BlackBoxOptim to save and load
#----------------------------------------------------------------------
if VERSION > v"0.6.4"
    using BlackBoxOptim
end

## Step 1: Initializing a SMMProblem

In [ ]:
myProblem = SMMProblem(options = SMMOptions(maxFuncEvals=1000, saveSteps = 1000, bbOptimizer = :dxnes));

## Step 2: Set emprical moments

In [ ]:
dictEmpiricalMoments = OrderedDict{String,Array{Float64,1}}()
dictEmpiricalMoments["mean"] = [1.0; 1.0]
dictEmpiricalMoments["variance"] = [5.0; 5.0]
dictEmpiricalMoments["autocorr"] = [sqrt(0.8); sqrt(0.8)]

In [ ]:
set_empirical_moments!(myProblem, dictEmpiricalMoments)

## Step 3. Set priors

In [ ]:
dictPriors = OrderedDict{String,Array{Float64,1}}()
dictPriors["delta"] = [0., -5.0, 5.0]
dictPriors["rho"] = [0., 0.01, 0.95]
dictPriors["sigma"] = [1., 0.01, 2.0]

One has to attach priors values the `SMMProblem` we are going to use.
This is achieved using the function  `set_priors!()`

In [ ]:
set_priors!(myProblem, dictPriors)

## Step 3: Specifying the function that generate simulated moments using parameter values

This function has two constraints:
* it should take as an argument a vector (the vector of parameters)
* it should return an OrderedDict containing the same moments as the ones contained in the OrderedDict 
containing empirical moments (`dictEmpiricalMoments`)

In this example, we simulate draws from a mutlivariate normal with unit variance:

In [ ]:
# x[1] corresponds to delta
# x[1] corresponds to rho
# x[3] corresponds to sigma
function functionAR1(x; nbDraws::Int64 = 1000000, burnInPerc::Int64 = 10)

    # Draw from standard normal:
    #--------------------------
    d = Normal()
    StdNormalDraws = rand(d, nbDraws)
    
    # Construct the AR(1) process:
    #-----------------------------
    seriesAR1 = zeros(StdNormalDraws)
    
    for t = 2:nbDraws
        seriesAR1[t] = x[1] + x[2]*seriesAR1[t-1] + x[3]*StdNormalDraws[t]
    end
    
    # Get rid of the burn-in phase:
    #------------------------------
    startT = div(nbDraws, burnInPerc)
    
    # Moments:
    #---------
    output = OrderedDict{String,Float64}()
    output["mean"] = mean(seriesAR1[startT:nbDraws])
    output["variance"] = var(seriesAR1[startT:nbDraws])
    output["autocorr"] = StatsBase.autocor(seriesAR1[startT:nbDraws], [1])[]

    return output
end

In [ ]:
functionAR1([1- sqrt(0.8); sqrt(0.8); 1.0])

In [ ]:
set_simulate_empirical_moments!(myProblem, functionAR1)

## Step 4. Constructing the objective function

In [ ]:
# Construct the objective function using: 
#* the function: parameter -> simulated moments
#* emprical moments values
#* emprical moments weights
construct_objective_function!(myProblem)

## Step 5. Running the optimization

In [ ]:
# Run the optimization
# This function first modifies t.bbSetup
# and then modifies t.bbResults
@time smmoptimize!(myProblem, verbose = true)

## Step 6. Analysing Results

### A. Point estimates

The parameter value that minimizes the distance between empirical and simulated moments can be accessed using 
the function `smm_minimizer` 

In [ ]:
minimizer = smm_minimizer(myProblem)
println("Estimated value for delta = $(minimizer[1])")
println("Estimated value for rho = $(minimizer[2])")
println("Estimated value for sigma = $(minimizer[3])")

### B. Standard errors

[WORKINPROGRESS]

### C. Slices


In [ ]:
Plots.plotly()
@time listPlots = smm_slices(myProblem, minimizer, 20)

You can combine the several plots created into a single one:

In [ ]:
Plots.plot(listPlots[1], listPlots[2], listPlots[3], leg = false)